In [3]:
from scipy.io import loadmat
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
matfile = loadmat("data/raw/Ach-AT/04_0316_ach-at_0.mat")

In [3]:
df = pd.DataFrame(matfile['filt_data'])
df=df.T
df_2 = pd.DataFrame()
df.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,7.400816,-5.906791,1.142726,0.191898,69.562051,9.012236,-1.880871,8.457310,8.753557,9.518978,...,10.667000,21.348741,24.569333,28.577441,-6.224571,-12.086969,13.948367,30.062192,16.205018,-2.557572
1,7.177488,-6.135068,0.915841,-0.035060,70.621536,8.793167,-2.098095,8.270087,8.722532,9.349439,...,10.660654,21.315009,24.523723,28.519099,-6.261806,-12.145811,13.811402,29.889178,16.011065,-2.787902
2,6.954183,-6.363301,0.688980,-0.262074,71.684292,8.574064,-2.315308,8.082819,8.691471,9.179862,...,10.654873,21.281652,24.478398,28.461005,-6.299013,-12.204617,13.674545,29.716228,15.817129,-3.018183
3,6.730903,-6.591484,0.462145,-0.489139,72.750299,8.354930,-2.532505,7.895511,8.660374,9.010250,...,10.649656,21.248670,24.433359,28.403161,-6.336192,-12.263387,13.537799,29.543346,15.623212,-3.248411
4,6.507654,-6.819615,0.235341,-0.716252,73.819535,8.135771,-2.749683,7.708165,8.629241,8.840605,...,10.645005,21.216063,24.388607,28.345568,-6.373342,-12.322119,13.401168,29.370533,15.429319,-3.478581


In [4]:
df_2 = pd.DataFrame()
for j in range(60):
    index = 0
    window = 1
    while index < 180000:
        # one row should contain 6000 voltage measurements (6s worth), the window number and the electrode number
        row = df.iloc[:,j]
        row = pd.DataFrame(row[index:index+6000])
        row = row.reset_index(drop=True)
        row = row.T
        row['electrode'] = j
        row['window'] = window
        df_2 = df_2.append(row)
        index += 6000
        window += 1

In [5]:
file = "data/raw/Ach-AT/04_0316_ach-at_0.mat"
name = os.path.split(file)[1]
y = name[-5]
df_2['y'] = y
df_2.head()
df_2['subject'] = name[:7]
df_2.to_hdf('test.h5', key = name[:7], mode = 'a')

In [6]:
test = pd.read_hdf('test.h5')

In [4]:
# load 
# get a list of the file names
d = "data/raw/Ach-AT"
filenames = []

for root, dirs, files in os.walk(d):
    for file in files:
        if file.endswith(".mat"):
            filenames.append(os.path.join(root, file))

for file in filenames:
    matfile = loadmat(file)
    df_raw = pd.DataFrame(matfile['filt_data'])
    df_raw = df_raw.T
    df_clean = pd.DataFrame()
    for j in range(60):
        index = 0
        window = 1
        while index < 180000:
            # one row should contain 6000 voltage measurements (6s worth), the window number and the electrode number
            row = df_raw.iloc[:,j]
            row = pd.DataFrame(row[index:index+6000])
            row = row.reset_index(drop=True)
            row = row.T
            row['electrode'] = j
            row['window'] = window
            df_clean = df_clean.append(row)
            index += 6000
            window += 1
    name = os.path.split(file)[1]
    y = name[-5]
    df_clean['y'] = y
    df_clean['subject'] = name[:7]
    df_clean.to_hdf('data/processed/ach_at_table.h5', key = 'Ach-AT', mode = 'a', format = 'table', append = True)